# Lets build a smarter AI App!

The following steps will cover:
- How to add data to your own chatbot using a text file.
- Use Azure Search to add data in a more sophisticated way.
- Create an improved web app that keeps track of the message history for improved context

# Part 1: Adding data from a text file

We can see how easy it is to add data to your AI search by just grabbing some text from a text file.

In this section, we'll create our own basic search function that looks through all the lines of the file and counts up how many keywords appear from a search. 

We'll return the 3 lines of hte file with the most matching kewyords and pass it to the AI to communocate to the user. 

## Get ready
1. Get your code from yesterday and paste it in the box below. Or use the box that is two down that I have preppared for you. 
    - You'll need to add your own key if you are using the provided code. 

2. Look at the data file we will be using, it's called pet_products.txt

3. update the question in your code to something relevant to the date, eg: "I have a puppy"

4. Below your imports add this contant. This provides a list of "Stop Words", things that don't help give useful search results.

    ```
    STOP_WORDS = set([
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", 
    "into", "is", "it", "no", "not", "of", "on", "or", "such", "that", "the", 
    "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"
    ])
    ```

5. After your STOP_WORDS (or anywhere before you create your messages for the AI), copy and paste this function. 
   This code looks at each line of the file and gives a point to the score for each word that matches the search question (but isn't a stop word). The 3 lines with the best score are returned as the search summary.
   ```
   def get_search_summary(question):
    all_keywords = set([word.lower() for word in question.split(" ")])
    # Remove stop words from the question
    keywords = all_keywords - STOP_WORDS
    
    # We'll score the scores based on the number of keywords in the line
    # eg: {"large dog bed": 2, "dog bed": 1}
    scores = {}
      
    # Open the file and read each line
    with open("pet_products.txt", "r") as f:
        for line in f:
            line = line.strip()
            # Make a list of numbers for each word in the line, 1 if the word is a keyword, 0 if not
            wordhits = [1 if word in line.lower() else 0 for word in keywords]
            # Add up the scores for the line
            line_score = sum(wordhits)
            # Only record the score if it's greater than 0
            if line_score > 0:
                scores[line] = line_score
            
    # Sort the scores, biggest to smalles, and get the top 3
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    search_summmary = [k for k, v in sorted_scores[:(min(3, len(sorted_scores)))]]
    return search_summmary
    ```

6. It's time to call the get_search_summary fucntion! Add this line of code before you create the message history. (Try printing it out to see what you got!)

    ```search_results = get_search_summary(question)```

7. Now turn the results from a list into a string on the next line.
   
   ```search_summary = " ".join(search_results)```

8. Change the last message in your messages list to look like this to include your search summary from the data.
   `{"role": "user", "content": question + "\nSources: " + search_summary},`

9. Give it a run!

In [2]:
# Imports go here
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

STOP_WORDS = set([
    "a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", 
    "into", "is", "it", "no", "not", "of", "on", "or", "such", "that", "the", 
    "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"
    ])

# Put the keys and endpoints here (never put your real keys in the code)
AOAI_ENDPOINT = "https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1"
AOAI_KEY = "90375df3-fbd6-4030-8d43-9d93938091e5"
MODEL_NAME = "shesharp-fp-hack-gpt35-turbo-16k"

# Search endpoints, etc
SEARCH_ENDPOINT = "https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1"
SEARCH_KEY = "90375df3-fbd6-4030-8d43-9d93938091e5"
AZURE_SEARCH_INDEX = "margiestraveldocs"

# Set up the client for AI Chat using the contstants and API Version
client = AzureOpenAI(
    api_key= AOAI_KEY,
    azure_endpoint= AOAI_ENDPOINT,
    api_version="2024-05-01-preview",
)

search_client = SearchClient(
    endpoint= SEARCH_ENDPOINT,
    credential=AzureKeyCredential(SEARCH_KEY),
    index_name=AZURE_SEARCH_INDEX
    )

# Set the tone of the conversation
SYSTEM_MESSAGE = "You are a helpful AI assistant that can answer questions and provide information, you are a surfer dude let us know it. You must use the provided sources."

def get_search_summary(question):
    all_keywords = set([word.lower() for word in question.split()])
    # Remove stop words from the question
    keywords = all_keywords - STOP_WORDS

    # We'll score the scores based on the number of keywords in the line
    # eg: {"large dog bed": 2, "dog bed": 1}
    sources = {}

    # Open the file and read each line
    with open("pet_products.txt", "r") as f:
        for line in f:
            line = line.strip()
            # Make a list of numbers for each word in the line, 1 if the word is a keyword, 0 if not
            wordhits = [1 if word in line.lower() else 0 for word in keywords]
            line_score = sum(wordhits)
            if line_score > 0:
                sources[line] = line_score

        sorted_scores = sorted(sources.items(), key=lambda x: x[1], reverse=True)
        search_summary = [k for k, v in sorted_scores[:(min(3, len(sorted_scores)))]]
        return search_summary

# Create the message history
messages=[
    {"role": "system", "content": SYSTEM_MESSAGE},
]

while True:
    # What question do you want to ask?
    # question = "I have a small animal, what can he sleep in?"
    question = input("What question do you want to ask? ")

    # search_results = get_search_summary(question)
    # search_summary = " ".join(search_results)

    search_results = search_client.search(search_text=question)
    search_summary = " ".join(result["content"] for result in search_results)    

    # Add the question to the message history
    messages.append({"role": "user", "content": question + "\nSources: " + search_summary})

    # Get the answer using the GPT model (create 1 answer (n) and use a temperature of 0.7 to set it to be pretty creative/random)
    response = client.chat.completions.create(model=MODEL_NAME,temperature=0.7,n=1,messages=messages)
    answer = response.choices[0].message.content
    print(answer)

    messages.append({"role": "system", "content": answer})

Dude, if you're looking for a place to stay in New York, Margie's Travel got you covered! They offer a few accommodation options in the city:

1. The Manhattan Hotel: This hotel is located in the heart of New York City, within walking distance of Times Square and Broadway.

2. The Grand Central Hotel: A comfortable midtown hotel, close to Grand Central Station, the Chrysler Building, and the Empire State Building.

3. The Park Hotel: This luxurious accommodation is situated in upper Manhattan, offering views of Central Park.

To book your trip to New York or check out other options, you can visit their website at www.margiestravel.com. Have a gnarly time in the Big Apple, dude!
Dude, if you're planning a trip to Dubai, Margie's Travel has got some rad accommodation options for you:

1. The Creek Hotel: This friendly boutique hotel is located in the heart of the bustling Dubai Creek area.

2. The Deira Hotel: A family-run hotel in Dubai's traditional commercial center.

3. The Lost City

KeyboardInterrupt: Interrupted by user

# Part 2: Add more sophisticated data to your chatbot with Azure Search
Our chatbot now knows how to read and to write and has some general purpose knowledge. But it doens't know about many specific resources or new resources (from after the model was trainied). We'll be adding some data that we can search through before it answers the question.

### 🌏♻️ Upate your environment
1. At the top of your code import the libraries need to **search documents** and create **login credentials for Azure** like this
    ```
    from azure.search.documents import SearchClient
    from azure.core.credentials import AzureKeyCredential
    ```

### 🔎 Add Search capabilties 
2. Add another constant called `SEARCH_ENDPOINT` and set it to the same things as your `AOAI_ENDPOINT` *(this is only the same because we are using the proxy service)*.

3. Add another constant called `SEARCH_KEY` and set it to the same things as your `AOAI_KEY`.

4. Add another constant called `AZURE_SEARCH_INDEX` and set it to `margiestraveldocs` to link to the pre-made Azure blob container *(I have uploaded relevant documents to the Azure Blob storage prior to the session so it will work with the proxy server)*:

5. Copy this code and put it under where the AI Chat Client is set up:

    ```
    search_client = SearchClient(
        endpoint=<PUT THE SEARCH ENDPOINT CONTSTANT HERE>,
        credential=AzureKeyCredential(<ADD SEARCH KEY CONSTANT HERE>),
        index_name=<PUT YOUR CHOSEN SEARCH INDEX HERE>,
        )
    ```
6. Put in the variable names for the Search Endpoint, Search Key, and Search Index.

### 🔎🧠 Use your AI Search Client
7. After you have asked the user for a question, put this bit of code to query the AI Search resource. 
     `search_results = search_client.search(search_text=question)`

8. On the next line add this code to join all of the results together into on big text chunk:
    `search_summary = " ".join(result["content"] for result in search_results)`

9. Update the messages variable so the user message looks like this (to include the search summary):
    `{"role": "user", "content": question + "\nSources: " + search_summary}`

11. Update your question to ask a question about travel. E.g. "where should I stay in New York?"

12. Update your SYSTEM_MESSAGE to say that "you must use provide resources"
    
13. Run your code and ask a question that relates to travel.


# Part 3.a - A prettier AI question answers
1. Add the following routes (under the other 3) to serve up the provided ask.html file with CSS. *(This uses JavaScript to return the form data in preparation for Part 4.)*
    ```
    @app.get('/ask')
    def ask():
        return render_template("ask.html")


    @app.route('/contextless-message', methods=['GET', 'POST'])
    def contextless_message():
        question = request.json['message']
        resp = get_response(question)
        return {"resp": resp[0]}
    ```
2. Run your server again and go to the /ask route on the server. 

## Part 3.b: Improve your app with a message history
In this part we've provided a HTML template, JavaScript and CSS that handle the front end to have an ongoing conversation. 
Each time we will unpack both the question and the previous chat history (the context) from the JSON we receive in the request. 

1. Add these two routes to your app.py file under your other routes.
    ```
    @app.get('/chat')
    def chat():
        return render_template('chat.html')

    @app.route("/context-message", methods=["GET", "POST"])
    def context_message():
        question = request.json["message"]
        context = request.json["context"]

        resp, context = get_response(question, context)
        return {"resp": resp, "context": context}
    ```

2. Update your get_response function where you create the message variable, get rid of your existing message variable and replace it with this if statement. *(This will test if there is already a message history to add to or if one should be created.)*
    ```
    # Create a new message history if there isn't one
    if not message_history:
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": question + "\nSources: " + search_summary},
        ]
    # Otherwise, append the user's question to the message history
    else:
        messages = message_history + [
            {"role": "user", "content": question + "\nSources: " + search_summary},
        ]
    ```

